In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [23]:
import sys, os
import numpy as np
import scipy 
import tensorflow as tf

import imageio
import gzip
from PIL import Image

import seaborn as sns
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
#from tqdm import trange, tqdm

import keras
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop, Optimizer
from keras.callbacks import Callback
from collections import OrderedDict

from helpers import utils
# from helpers import protocols
# from helpers.keras_utils import LossHistory
# from helpers.optimizers import KOOptimizer

In [24]:
tf.logging.set_verbosity(tf.logging.INFO)

EXTRACT LABELS

In [25]:
# Extract labels from MNIST labels into vector
def extract_labels(filename, num_images):
  with gzip.open(filename) as bytestream:
    bytestream.read(8)
    buf = bytestream.read(1 * num_images)
    labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  return labels

train_labels = extract_labels("MNIST-data/train-labels-idx1-ubyte.gz", 60000)
eval_labels = extract_labels("MNIST-data/t10k-labels-idx1-ubyte.gz", 10000)
print(np.shape(train_labels))
print(np.shape(eval_labels))

(60000,)
(10000,)


CONSTRUCT DATASETS


In [26]:
# original train
train_original = np.zeros((60000,28,28), dtype=np.float32)
images_original = ["MNIST-processed-training/original/original{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_original)):
    img = np.array(Image.open(images_original[i]))
    train_original[i, :, :] = img
 
print(np.shape(train_original))

# original test
eval_original = np.zeros((10000,28,28), dtype=np.float32)
images2_original = ["MNIST-processed-test/original/test-original{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_original)):
    img = np.array(Image.open(images2_original[i]))
    eval_original[i, :, :] = img
 
print(np.shape(eval_original))

(60000, 28, 28)
(10000, 28, 28)


In [27]:
# ROTATE 90 train
train_rot90 = np.zeros((60000,28,28), dtype=np.float32)
images_rot90 = ["MNIST-processed-training/rot90/rot90{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_rot90)):
    img = np.array(Image.open(images_rot90[i]))
    train_rot90[i, :, :] = img
 
print(np.shape(train_rot90))

# ROTATE 90 test
eval_rot90 = np.zeros((10000,28,28), dtype=np.float32)
images2_rot90 = ["MNIST-processed-test/rot90/test-rot90{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_rot90)):
    img = np.array(Image.open(images2_rot90[i]))
    eval_rot90[i, :, :] = img
 
print(np.shape(eval_rot90))

(60000, 28, 28)
(10000, 28, 28)


In [28]:
# checkerboard train
train_checkerboard = np.zeros((60000,28,28), dtype=np.float32)
images_checkerboard = ["MNIST-processed-training/checkerboard/fullcheck{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_checkerboard)):
    img = np.array(Image.open(images_checkerboard[i]))
    train_checkerboard[i, :, :] = img
 
print(np.shape(train_checkerboard))

# checkerboard test
eval_checkerboard = np.zeros((10000,28,28), dtype=np.float32)
images2_checkerboard = ["MNIST-processed-test/checkerboard/test-checkerboard{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_checkerboard)):
    img = np.array(Image.open(images2_checkerboard[i]))
    eval_checkerboard[i, :, :] = img
 
print(np.shape(eval_checkerboard))

(60000, 28, 28)
(10000, 28, 28)


In [29]:
# INV train
train_inv = np.zeros((60000,28,28), dtype=np.float32)
images_inv = ["MNIST-processed-training/Inv/inv{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_inv)):
    img = np.array(Image.open(images_inv[i]))
    train_inv[i, :, :] = img
 
print(np.shape(train_inv))

# INV test
eval_inv = np.zeros((10000,28,28), dtype=np.float32)
images2_inv = ["MNIST-processed-test/inv/test-inv{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_inv)):
    img = np.array(Image.open(images2_inv[i]))
    eval_inv[i, :, :] = img
 
print(np.shape(eval_inv))

(60000, 28, 28)
(10000, 28, 28)


In [30]:
# invbot train
train_invbot = np.zeros((60000,28,28), dtype=np.float32)
images_invbot = ["MNIST-processed-training/invbot/invbot{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_invbot)):
    img = np.array(Image.open(images_invbot[i]))
    train_invbot[i, :, :] = img
 
print(np.shape(train_invbot))

# invbot test
eval_invbot = np.zeros((10000,28,28), dtype=np.float32)
images2_invbot = ["MNIST-processed-test/invbot/test-invbot{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_invbot)):
    img = np.array(Image.open(images2_invbot[i]))
    eval_invbot[i, :, :] = img
 
print(np.shape(eval_invbot))

(60000, 28, 28)
(10000, 28, 28)


In [31]:
# fliplr train
train_fliplr = np.zeros((60000,28,28), dtype=np.float32)
images_fliplr = ["MNIST-processed-training/fliplr/fliplr{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_fliplr)):
    img = np.array(Image.open(images_fliplr[i]))
    train_fliplr[i, :, :] = img
 
print(np.shape(train_fliplr))

# fliplr test
eval_fliplr = np.zeros((10000,28,28), dtype=np.float32)
images2_fliplr = ["MNIST-processed-test/fliplr/test-fliplr{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_fliplr)):
    img = np.array(Image.open(images2_fliplr[i]))
    eval_fliplr[i, :, :] = img
 
print(np.shape(eval_fliplr))

(60000, 28, 28)
(10000, 28, 28)


In [32]:
# cutud train
train_cutud = np.zeros((60000,28,28), dtype=np.float32)
images_cutud = ["MNIST-processed-training/cutud/cutUD{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_cutud)):
    img = np.array(Image.open(images_cutud[i]))
    train_cutud[i, :, :] = img
 
print(np.shape(train_cutud))

# cutud test
eval_cutud = np.zeros((10000,28,28), dtype=np.float32)
images2_cutud = ["MNIST-processed-test/cutud/test-cutud{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_cutud)):
    img = np.array(Image.open(images2_cutud[i]))
    eval_cutud[i, :, :] = img
 
print(np.shape(eval_cutud))

(60000, 28, 28)
(10000, 28, 28)


In [34]:
# flipud train
train_flipud = np.zeros((60000,28,28), dtype=np.float32)
images_flipud = ["MNIST-processed-training/flipud/flipud{0}.png".format(k) for k in range(1,60000)]

for i in range(len(images_flipud)):
    img = np.array(Image.open(images_flipud[i]))
    train_flipud[i, :, :] = img
 
print(np.shape(train_flipud))

# flipud test
eval_flipud = np.zeros((10000,28,28), dtype=np.float32)
images2_flipud = ["MNIST-processed-test/flipud/test-flipud{0}.png".format(k) for k in range(1,10001)]

for i in range(len(images2_flipud)):
    img = np.array(Image.open(images2_flipud[i]))
    eval_flipud[i, :, :] = img
 
print(np.shape(eval_flipud))

(60000, 28, 28)
(10000, 28, 28)


PARAMETERS

In [35]:
# # Data params
# input_dim = 784
# output_dim = 10

# input image dimensions
img_rows, img_cols = 28, 28

# # Network params
# n_hidden_units = 2000
# activation_fn = tf.nn.relu

# Optimization params
batch_size = 256
num_classes = 10
epochs = 5 # epochs per task
# learning_rate=1e-3
# xi = 0.1

# Reset optimizer after each age
# reset_optimizer = False

In [37]:
# the data, train and test sets
x_train = train_original
x_test = eval_original
y_train = train_labels
y_test = eval_labels

print('train labels shape:', y_train.shape)
print('test labels shape:', y_test.shape)

# ?
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# ?
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('train labels shape:', y_train.shape)
print('test labels shape:', y_test.shape)

train labels shape: (60000,)
test labels shape: (10000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
train labels shape: (60000, 10)
test labels shape: (10000, 10)


In [38]:

x_train2 = train_rot90
x_test2 = eval_rot90
if K.image_data_format() == 'channels_first':
    x_train2 = x_train2.reshape(x_train2.shape[0], 1, img_rows, img_cols)
    x_test2 = x_test2.reshape(x_test2.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train2 = x_train2.reshape(x_train2.shape[0], img_rows, img_cols, 1)
    x_test2 = x_test2.reshape(x_test2.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train2 = x_train2.astype('float32')
x_test2 = x_test2.astype('float32')
x_train2 /= 255
x_test2 /= 255

x_train3 = train_inv
x_test3 = eval_inv
if K.image_data_format() == 'channels_first':
    x_train3 = x_train3.reshape(x_train3.shape[0], 1, img_rows, img_cols)
    x_test3 = x_test3.reshape(x_test3.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train3 = x_train3.reshape(x_train3.shape[0], img_rows, img_cols, 1)
    x_test3 = x_test3.reshape(x_test3.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train3 = x_train3.astype('float32')
x_test3 = x_test3.astype('float32')
x_train3 /= 255
x_test3 /= 255

x_train4 = train_flipud
x_test4 = eval_flipud
if K.image_data_format() == 'channels_first':
    x_train4 = x_train4.reshape(x_train4.shape[0], 1, img_rows, img_cols)
    x_test4 = x_test4.reshape(x_test4.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train4 = x_train4.reshape(x_train4.shape[0], img_rows, img_cols, 1)
    x_test4 = x_test4.reshape(x_test4.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train4 = x_train4.astype('float32')
x_test4 = x_test4.astype('float32')
x_train4 /= 255
x_test4 /= 255

x_train5 = train_fliplr
x_test5 = eval_fliplr
if K.image_data_format() == 'channels_first':
    x_train5 = x_train5.reshape(x_train5.shape[0], 1, img_rows, img_cols)
    x_test5 = x_test5.reshape(x_test5.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train5 = x_train5.reshape(x_train5.shape[0], img_rows, img_cols, 1)
    x_test5 = x_test5.reshape(x_test5.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train5 = x_train5.astype('float32')
x_test5 = x_test5.astype('float32')
x_train5 /= 255
x_test5 /= 255

x_train6 = train_cutud
x_test6 = eval_cutud
if K.image_data_format() == 'channels_first':
    x_train6 = x_train6.reshape(x_train6.shape[0], 1, img_rows, img_cols)
    x_test6 = x_test6.reshape(x_test6.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train6 = x_train6.reshape(x_train6.shape[0], img_rows, img_cols, 1)
    x_test6 = x_test6.reshape(x_test6.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train6 = x_train6.astype('float32')
x_test6 = x_test6.astype('float32')
x_train6 /= 255
x_test6 /= 255

x_train7 = train_invbot
x_test7 = eval_invbot
if K.image_data_format() == 'channels_first':
    x_train7 = x_train7.reshape(x_train7.shape[0], 1, img_rows, img_cols)
    x_test7 = x_test7.reshape(x_test7.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train7 = x_train7.reshape(x_train7.shape[0], img_rows, img_cols, 1)
    x_test7 = x_test7.reshape(x_test7.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train7 = x_train7.astype('float32')
x_test7 = x_test7.astype('float32')
x_train7 /= 255
x_test7 /= 255

x_train8 = train_checkerboard
x_test8 = eval_checkerboard
if K.image_data_format() == 'channels_first':
    x_train8 = x_train8.reshape(x_train8.shape[0], 1, img_rows, img_cols)
    x_test8 = x_test8.reshape(x_test8.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train8 = x_train8.reshape(x_train8.shape[0], img_rows, img_cols, 1)
    x_test8 = x_test8.reshape(x_test8.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
x_train8 = x_train8.astype('float32')
x_test8 = x_test8.astype('float32')
x_train8 /= 255
x_test8 /= 255



CONSTRUCT MODEL

In [39]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


TRAIN MODEL

In [40]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save trained model
model.save('trained_model.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 184s 3ms/step - loss: 0.3585 - acc: 0.8888 - val_loss: 0.0795 - val_acc: 0.9745
Epoch 2/5
60000/60000 [==============================] - 177s 3ms/step - loss: 0.1031 - acc: 0.9697 - val_loss: 0.0547 - val_acc: 0.9817
Epoch 3/5
60000/60000 [==============================] - 178s 3ms/step - loss: 0.0762 - acc: 0.9772 - val_loss: 0.0383 - val_acc: 0.9871
Epoch 4/5
60000/60000 [==============================] - 184s 3ms/step - loss: 0.0614 - acc: 0.9814 - val_loss: 0.0417 - val_acc: 0.9861
Epoch 5/5
60000/60000 [==============================] - 187s 3ms/step - loss: 0.0545 - acc: 0.9827 - val_loss: 0.0336 - val_acc: 0.9888
Test loss: 0.033603799838290434
Test accuracy: 0.9888


In [41]:
# model2 = Sequential()

# model2.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model2.add(Conv2D(64, (3, 3), activation='relu'))
# model2.add(MaxPooling2D(pool_size=(2, 2)))
# model2.add(Dropout(0.25))
# model2.add(Flatten())
# model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.5))
# model2.add(Dense(num_classes, activation='softmax'))
# model2.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])


# Reload model
model2 = load_model('trained_model.h5')

# Continue training
model2.fit(x_train2, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model2.evaluate(x_test, y_test, verbose=0)
score2 = model2.evaluate(x_test2, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('Second data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

# Save trained model
model2.save('trained_model2.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 183s 3ms/step - loss: 0.3223 - acc: 0.9047 - val_loss: 0.6549 - val_acc: 0.7928
Epoch 2/5
60000/60000 [==============================] - 197s 3ms/step - loss: 0.1119 - acc: 0.9674 - val_loss: 1.1198 - val_acc: 0.6905
Epoch 3/5
60000/60000 [==============================] - 177s 3ms/step - loss: 0.0842 - acc: 0.9754 - val_loss: 1.7272 - val_acc: 0.6029
Epoch 4/5
60000/60000 [==============================] - 171s 3ms/step - loss: 0.0699 - acc: 0.9793 - val_loss: 2.4287 - val_acc: 0.5139
Epoch 5/5
60000/60000 [==============================] - 164s 3ms/step - loss: 0.0602 - acc: 0.9824 - val_loss: 2.7129 - val_acc: 0.4359
Original data set
Test loss: 2.712919066810608
Test accuracy: 0.4359
Second data set
Test loss: 0.036111295583665194
Test accuracy: 0.9877


In [43]:
# model3 = Sequential()

# model3.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model3.add(Conv2D(64, (3, 3), activation='relu'))
# model3.add(MaxPooling2D(pool_size=(2, 2)))
# model3.add(Dropout(0.25))
# model3.add(Flatten())
# model3.add(Dense(128, activation='relu'))
# model3.add(Dropout(0.5))
# model3.add(Dense(num_classes, activation='softmax'))
# model3.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# Reload model
model3 = load_model('trained_model2.h5')

# Continue training
model3.fit(x_train3, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model3.evaluate(x_test, y_test, verbose=0)
score2 = model3.evaluate(x_test2, y_test, verbose=0)
score3 = model3.evaluate(x_test3, y_test, verbose=0)


print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('Second data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

print('Second data set')
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

# Save trained model
model3.save('trained_model3.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 157s 3ms/step - loss: 0.2013 - acc: 0.9419 - val_loss: 0.3987 - val_acc: 0.8580
Epoch 2/5
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0849 - acc: 0.9741 - val_loss: 0.7368 - val_acc: 0.7565
Epoch 3/5
60000/60000 [==============================] - 154s 3ms/step - loss: 0.0657 - acc: 0.9805 - val_loss: 0.5341 - val_acc: 0.8086
Epoch 4/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0539 - acc: 0.9839 - val_loss: 0.7304 - val_acc: 0.7647
Epoch 5/5
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0468 - acc: 0.9859 - val_loss: 0.7390 - val_acc: 0.7626
Original data set
Test loss: 0.7389654237508774
Test accuracy: 0.7626
Second data set
Test loss: 2.562846361351013
Test accuracy: 0.4234
Second data set
Test loss: 0.03205857433127967
Test accuracy: 0.9896


In [44]:
# model4 = Sequential()

# model4.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model4.add(Conv2D(64, (3, 3), activation='relu'))
# model4.add(MaxPooling2D(pool_size=(2, 2)))
# model4.add(Dropout(0.25))
# model4.add(Flatten())
# model4.add(Dense(128, activation='relu'))
# model4.add(Dropout(0.5))
# model4.add(Dense(num_classes, activation='softmax'))
# model4.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# Reload model
model4 = load_model('trained_model3.h5')

# Continue training
model4.fit(x_train4, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model4.evaluate(x_test, y_test, verbose=0)
score2 = model4.evaluate(x_test2, y_test, verbose=0)
score3 = model4.evaluate(x_test3, y_test, verbose=0)
score4 = model4.evaluate(x_test4, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('2 data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

print('3 data set')
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

print('4 data set')
print('Test loss:', score4[0])
print('Test accuracy:', score4[1])

# Save trained model
model4.save('trained_model4.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.3044 - acc: 0.9121 - val_loss: 3.2784 - val_acc: 0.2707
Epoch 2/5
60000/60000 [==============================] - 157s 3ms/step - loss: 0.1042 - acc: 0.9696 - val_loss: 3.8944 - val_acc: 0.2293
Epoch 3/5
60000/60000 [==============================] - 151s 3ms/step - loss: 0.0801 - acc: 0.9761 - val_loss: 4.1714 - val_acc: 0.2181
Epoch 4/5
60000/60000 [==============================] - 154s 3ms/step - loss: 0.0644 - acc: 0.9802 - val_loss: 5.4578 - val_acc: 0.1811
Epoch 5/5
60000/60000 [==============================] - 149s 2ms/step - loss: 0.0570 - acc: 0.9830 - val_loss: 5.4171 - val_acc: 0.1953
Original data set
Test loss: 5.417119498443603
Test accuracy: 0.1953
2 data set
Test loss: 5.454559282684326
Test accuracy: 0.2267
3 data set
Test loss: 3.50109836730957
Test accuracy: 0.483
4 data set
Test loss: 0.028514384727807193
Test accuracy: 0.9906


In [48]:
# model5 = Sequential()
# model5.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model5.add(Conv2D(64, (3, 3), activation='relu'))
# model5.add(MaxPooling2D(pool_size=(2, 2)))
# model5.add(Dropout(0.25))
# model5.add(Flatten())
# model5.add(Dense(128, activation='relu'))
# model5.add(Dropout(0.5))
# model5.add(Dense(num_classes, activation='softmax'))
# model5.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# Reload model
model5 = load_model('trained_model4.h5')

# Continue training
model5.fit(x_train5, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model5.evaluate(x_test, y_test, verbose=0)
score2 = model5.evaluate(x_test2, y_test, verbose=0)
score3 = model5.evaluate(x_test3, y_test, verbose=0)
score4 = model5.evaluate(x_test4, y_test, verbose=0)
score5 = model5.evaluate(x_test5, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('2 data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

print('3 data set')
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

print('4 data set')
print('Test loss:', score4[0])
print('Test accuracy:', score4[1])


print('5 data set')
print('Test loss:', score5[0])
print('Test accuracy:', score5[1])

# Save trained model
model5.save('trained_model5.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 183s 3ms/step - loss: 0.3263 - acc: 0.9058 - val_loss: 6.6734 - val_acc: 0.1322
Epoch 2/5
60000/60000 [==============================] - 180s 3ms/step - loss: 0.1090 - acc: 0.9681 - val_loss: 5.7889 - val_acc: 0.1702
Epoch 3/5
60000/60000 [==============================] - 179s 3ms/step - loss: 0.0822 - acc: 0.9765 - val_loss: 6.9074 - val_acc: 0.1506
Epoch 4/5
60000/60000 [==============================] - 178s 3ms/step - loss: 0.0698 - acc: 0.9797 - val_loss: 6.8747 - val_acc: 0.1660
Epoch 5/5
60000/60000 [==============================] - 172s 3ms/step - loss: 0.0607 - acc: 0.9824 - val_loss: 8.6029 - val_acc: 0.1096
Original data set
Test loss: 8.602926930236816
Test accuracy: 0.1096
2 data set
Test loss: 4.607483555603027
Test accuracy: 0.273
3 data set
Test loss: 4.280151822662353
Test accuracy: 0.4227
4 data set
Test loss: 5.093793195343018
Test accuracy: 0.4807
5 data set


In [49]:
# model6 = Sequential()
# model6.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model6.add(Conv2D(64, (3, 3), activation='relu'))
# model6.add(MaxPooling2D(pool_size=(2, 2)))
# model6.add(Dropout(0.25))
# model6.add(Flatten())
# model6.add(Dense(128, activation='relu'))
# model6.add(Dropout(0.5))
# model6.add(Dense(num_classes, activation='softmax'))
# model6.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# Reload model
model5 = load_model('trained_model4.h5')

# Continue training
model5.fit(x_train5, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model5.evaluate(x_test, y_test, verbose=0)
score2 = model5.evaluate(x_test2, y_test, verbose=0)
score3 = model5.evaluate(x_test3, y_test, verbose=0)
score4 = model5.evaluate(x_test4, y_test, verbose=0)
score5 = model5.evaluate(x_test5, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('2 data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

print('3 data set')
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

print('4 data set')
print('Test loss:', score4[0])
print('Test accuracy:', score4[1])


print('5 data set')
print('Test loss:', score5[0])
print('Test accuracy:', score5[1])

# Save trained model
model5.save('trained_model5.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 168s 3ms/step - loss: 0.3295 - acc: 0.9045 - val_loss: 5.6291 - val_acc: 0.1603
Epoch 2/5
60000/60000 [==============================] - 180s 3ms/step - loss: 0.1117 - acc: 0.9677 - val_loss: 7.3592 - val_acc: 0.1332
Epoch 3/5
60000/60000 [==============================] - 154s 3ms/step - loss: 0.0835 - acc: 0.9753 - val_loss: 7.6739 - val_acc: 0.1436
Epoch 4/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0686 - acc: 0.9794 - val_loss: 7.0773 - val_acc: 0.1450
Epoch 5/5
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0617 - acc: 0.9822 - val_loss: 8.1081 - val_acc: 0.1136
Original data set
Test loss: 8.108143403625489
Test accuracy: 0.1136
2 data set
Test loss: 4.581724743652344
Test accuracy: 0.2402
3 data set
Test loss: 3.5188808303833006
Test accuracy: 0.4677
4 data set
Test loss: 4.375547528076172
Test accuracy: 0.4731
5 data se

In [51]:
# Reload model
model6 = load_model('trained_model5.h5')

# Continue training
model6.fit(x_train6, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score1 = model6.evaluate(x_test, y_test, verbose=0)
score2 = model6.evaluate(x_test2, y_test, verbose=0)
score3 = model6.evaluate(x_test3, y_test, verbose=0)
score4 = model6.evaluate(x_test4, y_test, verbose=0)
score5 = model6.evaluate(x_test5, y_test, verbose=0)
score6 = model6.evaluate(x_test6, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('2 data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

print('3 data set')
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

print('4 data set')
print('Test loss:', score4[0])
print('Test accuracy:', score4[1])


print('5 data set')
print('Test loss:', score5[0])
print('Test accuracy:', score5[1])

print('6 data set')
print('Test loss:', score6[0])
print('Test accuracy:', score6[1])


# Save trained model
model6.save('trained_model6.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 182s 3ms/step - loss: 0.4091 - acc: 0.8797 - val_loss: 9.4218 - val_acc: 0.1602
Epoch 2/5
60000/60000 [==============================] - 159s 3ms/step - loss: 0.1331 - acc: 0.9605 - val_loss: 10.5342 - val_acc: 0.1369
Epoch 3/5
60000/60000 [==============================] - 151s 3ms/step - loss: 0.1022 - acc: 0.9704 - val_loss: 8.1295 - val_acc: 0.1778
Epoch 4/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0838 - acc: 0.9752 - val_loss: 8.7704 - val_acc: 0.1769
Epoch 5/5
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0724 - acc: 0.9785 - val_loss: 9.4676 - val_acc: 0.1624
Original data set
Test loss: 9.467570140075683
Test accuracy: 0.1624
2 data set
Test loss: 5.590687184143066
Test accuracy: 0.216
3 data set
Test loss: 3.4266992835998535
Test accuracy: 0.5322
4 data set
Test loss: 2.7531645259857176
Test accuracy: 0.5787
5 data s

In [52]:
# Reload model
model7 = load_model('trained_model6.h5')

# Continue training
model7.fit(x_train7, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1,
          validation_data=(x_test, y_test))
score = model6.evaluate(x_test, y_test, verbose=0)
score2 = model6.evaluate(x_test2, y_test, verbose=0)
score3 = model6.evaluate(x_test3, y_test, verbose=0)
score4 = model6.evaluate(x_test4, y_test, verbose=0)
score5 = model6.evaluate(x_test5, y_test, verbose=0)
score6 = model6.evaluate(x_test6, y_test, verbose=0)
score7 = model6.evaluate(x_test7, y_test, verbose=0)

print('Original data set')
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

print('2 data set')
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])

print('3 data set')
print('Test loss:', score3[0])
print('Test accuracy:', score3[1])

print('4 data set')
print('Test loss:', score4[0])
print('Test accuracy:', score4[1])


print('5 data set')
print('Test loss:', score5[0])
print('Test accuracy:', score5[1])

print('6 data set')
print('Test loss:', score6[0])
print('Test accuracy:', score6[1])

print('7 data set')
print('Test loss:', score7[0])
print('Test accuracy:', score7[1])


# Save trained model
model7.save('trained_model7.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 157s 3ms/step - loss: 0.2613 - acc: 0.9309 - val_loss: 0.4489 - val_acc: 0.8767
Epoch 2/5
60000/60000 [==============================] - 154s 3ms/step - loss: 0.0899 - acc: 0.9730 - val_loss: 0.4068 - val_acc: 0.8979
Epoch 3/5
60000/60000 [==============================] - 155s 3ms/step - loss: 0.0708 - acc: 0.9792 - val_loss: 0.3240 - val_acc: 0.9127
Epoch 4/5
60000/60000 [==============================] - 149s 2ms/step - loss: 0.0602 - acc: 0.9820 - val_loss: 0.3660 - val_acc: 0.9073
Epoch 5/5
60000/60000 [==============================] - 150s 2ms/step - loss: 0.0533 - acc: 0.9843 - val_loss: 0.3952 - val_acc: 0.9059
Original data set
Test loss: 9.467570140075683
Test accuracy: 0.1624
2 data set
Test loss: 5.590687184143066
Test accuracy: 0.216
3 data set
Test loss: 3.4266992835998535
Test accuracy: 0.5322
4 data set
Test loss: 2.7531645259857176
Test accuracy: 0.5787
5 data se

In [ ]:
def cnn_model_fn(features, labels, mode):

    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional layer 2 and pooling layer
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu
    )

    # Pooling 2 with flattening
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    pool2_flat=tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense layer with dropout 
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout=tf.layers.dropout(inputs=dense, rate=0.4, training=mode==tf.estimator.ModeKeys.TRAIN)
    logits = tf.layers.dense(inputs=dropout, units=10)

    # Generate predictions
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Caluclate loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure training op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
# Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Logging predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, 
    every_n_iter=50)

# Our application logic will be added here

In [ ]:
def main(unused_argv):
    # Training on original
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":train_original},
        y=train_labels,
        batch_size=1000,
        num_epochs=None,
        shuffle=True)
    
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluation on original
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_original},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    
    eval_results=mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == "__main__":
  tf.app.run()